# Scraping Congressional Travel Reports


The house of representatives publishes annual reports on Congressional travel. The tough part is sorting out all the different spellings--England, the UK,United Kingdom, etc.In this worksheet we're going to download those files, arrange them in a DataFrame, and use a homemade function to sort out the international destinations.

In [29]:
import wget
import requests
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
from Andrew import sandwich
import numpy as np
import os
import time
import glob

Start with a few variables that will help identify international trips

In [30]:
#List of states and other destinations within the United States
list_of_states='''
Alabama	AL	Ala.
Alaska	AK	Alaska
Arizona	AZ	Ariz.
Arkansas	AR	Ark.
California	CA	Calif.
Colorado	CO	Colo.
Connecticut	CT	Conn.
Delaware	DE	Del.
Florida	FL	Fla.
Georgia	GA	Ga.
Hawaii	HI	Hawaii
Idaho	ID	Idaho
Illinois	IL	Ill.
Indiana	IN	Ind.
Iowa	IA	Iowa
Kansas	KS	Kans.
Kentucky	KY	Ky.
Louisiana	LA	La.
Maine	ME	Maine
Maryland	MD	Md.
Massachusetts	MA	Mass.
Michigan	MI	Mich.
Minnesota	MN	Minn.
Mississippi	MS	Miss.
Missouri	MO	Mo.
Montana	MT	Mont.
Nebraska	NE	Neb. or Nebr.
Nevada	NV	Nev.
New Hampshire	NH	N.H.
New Jersey	NJ	N.J.
New Mexico	NM	N.Mex.
New York	NY	N.Y.
North Carolina	NC	N.C.
North Dakota	ND	N.Dak.
Ohio	OH	Ohio
Oklahoma	OK	Okla.
Oregon	OR	Ore. or Oreg.
Pennsylvania	PA	Pa.
Rhode Island	RI	R.I.
South Carolina	SC	S.C.
South Dakota	SD	S.Dak.
Tennessee	TN	Tenn.
Texas	TX	Tex. or Texas
Utah	UT	Utah
Vermont	VT	Vt.
Virginia	VA	Va.
Washington	WA	Wash.
West Virginia	WV	W.Va.
Wisconsin	WI	Wis. or Wisc.
Wyoming	WY	Wyo.
District of Columbia\tD.C.\tDC
NYC
New York
Los Angeles
Baltimore
Cambridge
Hot Springs
Williamsburg
Las Vegas
Philadelphia
Warrenton
Middleburg
San Francisco
Miami
New Orleans
Seattle
Palm Beach
Orlando
Clewiston
Stanford
Nashville
White Sulfur Springs
St Louis
Aspen
Fort Lauderdale
Pikesville
Williamsburg
Puerto Rico
Boston
San Francisco
Tampa
Miami'''
list_of_states=list_of_states.replace('\n','\t').replace('.','').split('\t')
list_of_states.remove('')

In [31]:
list_of_countries='''Afghanistan
Albania
Algeria
Andorra
Angola
Antigua and Barbuda
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Central African Republic
CAR
Chad
Chile
China
PRC
Colombia
Comoros
Congo
DRC
Costa Rica
Côte d’Ivoire
Ivory Coast
Croatia
Cuba
Cyprus
Czech Republic
Czechia
Denmark
Djibouti
Dominica
Dominican Republic
East Timor
Timor-Leste
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Fiji
Finland
France
Gabon
Gambia
Germany
Ghana
Greece
Grenada
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kiribati
North Korea
DPRK
South Korea
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Liechtenstein
Lithuania
Luxembourg
Macedonia
Madagascar
Malawi
Malaysia
Maldives
Mali
Malta
Marshall Islands
Mauritania
Mauritius
Mexico
Micronesia
Moldova
Monaco
Mongolia
Montenegro
Morocco
Mozambique
Myanmar
Burma
Namibia
Nauru
Nepal
Netherlands
New Zealand
Nicaragua
Niger
Nigeria
North Macedonia
Norway
Oman
Pakistan
Palau
Panama
Papua New Guinea
Paraguay
Peru
Philippines
Poland
Portugal
Qatar
Romania
Russia
Rwanda
Saint Kitts and Nevis
Saint Lucia
Saint Vincent and the Grenadines
Samoa
San Marino
Sao Tome and Principe
Saudi Arabia
Senegal
Serbia
Seychelles
Sierra Leone
Singapore
Slovakia
Slovenia
Solomon Islands
Somalia
South Africa
Spain
Sri Lanka
Sudan
South Sudan
Suriname
Sweden
Switzerland
Syria
Taiwan
Tajikistan
Tanzania
Thailand
Togo
Tonga
Trinidad and Tobago
Tunisia
Turkey
Turkiye
Turkmenistan
Tuvalu
Uganda
Ukraine
United Arab Emirates
UAE
United Kingdom
UK
Britain
Uruguay
Uzbekistan
Vanuatu
Vatican City
Venezuela
Vietnam
Yemen
Zambia
Zimbabwe'''
list_of_countries=list_of_countries
list_of_countries=list_of_countries.split('\n')
list_of_countries

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'CAR',
 'Chad',
 'Chile',
 'China',
 'PRC',
 'Colombia',
 'Comoros',
 'Congo',
 'DRC',
 'Costa Rica',
 'Côte d’Ivoire',
 'Ivory Coast',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Czechia',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Timor-Leste',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'H

In [32]:
Israel_destinations=["israel","tel avi", "jerusalem", "tiberias","tiberas","tiberius", 
                 "isreal", "tibenus",
    "tel-aviv", "israe", "sderot", "jerusalum", "ben gurion", "jersalem", "neve shalom",
     "tiberus"]#some filers just list the city, so it's important to identify every possible 
#way of saying the same thing. 

In [33]:
Palestine_destinations=["west bank","gaza","ramallah","nablus","jenin", "palestin"]


In [47]:
def findcollisions(mystring, mylist):
    '''
    searches a string 'mystring' for elements listed in [mylist]. Returns all 
    elements separated by a space
    '''
    stringlist=mystring.split(',') #changes mystring to list form, splitting at commas
    text=""
    for item in mylist:
        if item in mystring:
            text=text+" "+item
    return text.strip()

Using the previous variables/functions, the above is used to determine which country a particular trip visited. This is particularly important for omitting trips within the US.

In [48]:
def identify_country(string):
    '''identifies countries based on the established lists of countries and states.
    assumes that no string has locations in both the US and a foreign country. 
    If the function cannot identify the destination country based on those lists, it returns 
    the original string
    '''
    string.replace("Britain","United Kingdom").replace("UK","United Kingdom").replace("England", "United Kingdom")
    string.replace("Ottawa","Canada").replace("Calgary","Canada")
    string.replace("Columbia","Colombia") #This section removes some common misspellings.
    string.replace("United Arab Emirates","UAE")
    if any(x in string.lower() for x in Israel_destinations):
        return "Israel"
    elif any(x in string.lower() for x in Palestine_destinations):
        return "Palestine"
    elif any(x in string for x in list_of_countries):
        return findcollisions(string, list_of_countries)
    elif any(x in string for x in list_of_states):
        return "USA"
    else:
        return string

Let's see if they work

In [49]:
print(identify_country("Michigan"))
print(identify_country("Ann Arbor, MI"))
print(identify_country("Paris France"))
print(identify_country("Tel Aviv, Israel"))
print(identify_country("Democratic Republic of C"))

USA
USA
France
Israel
Democratic Republic of C


The files we want are listed here:https://disclosures-clerk.house.gov/GiftTravelFilings
I have downloaded the last five years' worth.

In [51]:
housefiles=glob.glob('../../../Pitches/*.txt')
housefiles

['../../../Pitches/2023Travel.txt',
 '../../../Pitches/2022Travel.txt',
 '../../../Pitches/2021Travel.txt',
 '../../../Pitches/2019Travel.txt',
 '../../../Pitches/2020Travel.txt']

In [52]:
dfs=[]
for file in housefiles:
    df=pd.read_table('../../../Pitches/2023Travel.txt', delimiter='\t')
    dfs.concat([dfs,df])
dfs

AttributeError: 'list' object has no attribute 'concat'

In [62]:
travel2023=pd.read_table('../../../Pitches/2023Travel.txt', delimiter='\t')
travel2023

,DocID,FilerName,MemberName,State,District,Year,Destination,FilingType,DepartureDate,ReturnDate,TravelSponsor
0,500025753,Sierra Kelley-Chung,"Horsford, Steven",NV,4.0,2023.0,"Las Vegas, NV",Original,1/6/2023,1/8/2023,Consumer Technology Association
1,500025755,Chloe Hunt,"Casten, Sean",IL,6.0,2023.0,"Las Vegas, NV",Original,1/6/2023,1/8/2023,Consumer Technology Association
2,500025756,Elizabeth Fallon,"Emmer, Tom",MN,6.0,2023.0,"Las Vegas, NV",Original,1/5/2023,1/7/2023,Consumer Technology Association
3,500025757,Michael Cameron,"Rodgers, Cathy",WA,5.0,2023.0,"Las Vegas, NV",Original,1/4/2023,1/6/2023,Consumer Technology Association
4,500025758,Shana Teehan,"Moore, Barry",AL,2.0,2023.0,"Las Vegas, NV",Original,1/6/2023,1/8/2023,Consumer Technology Association
...,...,...,...,...,...,...,...,...,...,...,...
1733,500027330,Derron Bennett,"Wilson, Frederica",FL,24.0,2024.0,"Thibodaux, LA",Original,11/9/2023,11/11/2023,"Louisiana Sugar Cane Foundation, Inc."
1734,500027332,Allen Klump,"Duncan, Jeff",SC,3.0,2024.0,United Arab Emirates,Original,12/6/2023,12/11/2023,Conservative Climate Foundation
1735,500027333,Thomas Reynolds,"Carter, Earl",GA,1.0,2024.0,United Arab Emirates,Original,12/6/2023,12/11/2023,Conservative Climate Foundation
1736,500027334,Alexandria Fulling,"Walberg, Tim",MI,5.0,2024.0,United Arab Emirates,Original,12/6/2023,12/11/2023,Conservative Climate Foundation


In [60]:
#I already downloaded 2023 so we only need these 4:
file2022='../../../Pitches/2022Travel.txt'
file2021='../../../Pitches/2021Travel.txt'
file2019='../../../Pitches/2019Travel.txt'
file2020='../../../Pitches/2020Travel.txt'

In [61]:
travel2019=pd.read_table(file2019, delimiter='\t')
travel2020=pd.read_table(file2020, delimiter='\t')
travel2021=pd.read_table(file2021, delimiter='\t')
travel2022=pd.read_table(file2022, delimiter='\t')

In [63]:
#Let's make a dataframe to concatenate all of these datasets.
df=pd.concat([travel2019,travel2020,travel2021,travel2022,travel2023])

In [64]:
df

,DocID,FilerName,MemberName,State,District,Year,Destination,FilingType,DepartureDate,ReturnDate,TravelSponsor
0,500019474,Amy Bos,"Sensenbrenner, F. James",WI,5.0,2018.0,"Redmond, WA",Original,5/2/2019,5/4/2019,Microsoft
1,500020625,Austin Weatherford,"Kinzinger, Adam",IL,16.0,2019.0,"Las Vegas, NV",Original,1/7/2019,1/9/2019,Consumer Electronics Association
2,500020626,Davis Pace,"Ratcliffe, John",TX,4.0,2019.0,"Las Vegas, NV",Original,1/9/2019,1/11/2019,Consumer Technology Association
3,500020627,Yuri Beckelman,"Takano, Mark",CA,39.0,2019.0,"Las Vegas, NV",Original,1/8/2019,1/10/2019,Consumer Technology Association
4,500020628,Benjamin Elleson,"Long, Billy",MO,7.0,2019.0,"Las Vegas, NV",Original,1/9/2019,1/11/2019,Consumer Technology Association
...,...,...,...,...,...,...,...,...,...,...,...
1733,500027330,Derron Bennett,"Wilson, Frederica",FL,24.0,2024.0,"Thibodaux, LA",Original,11/9/2023,11/11/2023,"Louisiana Sugar Cane Foundation, Inc."
1734,500027332,Allen Klump,"Duncan, Jeff",SC,3.0,2024.0,United Arab Emirates,Original,12/6/2023,12/11/2023,Conservative Climate Foundation
1735,500027333,Thomas Reynolds,"Carter, Earl",GA,1.0,2024.0,United Arab Emirates,Original,12/6/2023,12/11/2023,Conservative Climate Foundation
1736,500027334,Alexandria Fulling,"Walberg, Tim",MI,5.0,2024.0,United Arab Emirates,Original,12/6/2023,12/11/2023,Conservative Climate Foundation


In [65]:
#We're going to need a dictionary to map different destinations to countries
destinations=df["Destination"].value_counts()
mydict={np.NaN:np.NaN}
for item in destinations.index:
    mydict[item]=identify_country(item)
mydict

{nan: nan,
 'Williamsburg, VA': 'USA',
 'Hot Springs, VA': 'USA',
 'Cambridge, MD': 'USA',
 'Tel Aviv, Israel': 'Israel',
 'New York, NY': 'USA',
 'Israel': 'Israel',
 'Baltimore, MD': 'USA',
 'Las Vegas, NV': 'USA',
 'Jerusalem, Israel': 'Israel',
 'Middleburg, VA': 'USA',
 'Tiberias, Israel': 'Israel',
 'Warrenton, VA': 'USA',
 'White Sulpher Springs, WV': 'USA',
 'Stanford, CA': 'USA',
 'San Francisco, CA': 'USA',
 'West Palm Beach, FL': 'USA',
 'New Orleans, LA': 'USA',
 'Boston, MA': 'USA',
 'Richmond, VA': 'USA',
 'Miami, FL': 'USA',
 'Clewiston, FL': 'USA',
 'Cambridge, MA': 'USA',
 'Des Moines, IA': 'USA',
 'United Kingdom': 'United Kingdom',
 'Tokyo, Japan': 'Japan',
 'Japan': 'Japan',
 'Chicago, IL': 'USA',
 'Carroll, IA': 'USA',
 'Orlando, FL': 'USA',
 'Seattle, WA': 'USA',
 'San Diego, CA': 'USA',
 'Los Angeles, CA': 'USA',
 'Omaha, NE': 'USA',
 'Amelia Island, FL': 'USA',
 'Philadelphia, PA': 'USA',
 'Thibodaux, LA': 'USA',
 'Brussels, Belgium': 'Belgium',
 'Berlin, German

In [39]:
mydict[np.nan]

nan

In [74]:
df["Country"]=df["Destination"].apply(lambda x: mydict[x])
travel2023["Country"]=travel2023["Destination"].apply(lambda x: mydict[x])
travel2022["Country"]=travel2022["Destination"].apply(lambda x: mydict[x])
travel2021["Country"]=travel2021["Destination"].apply(lambda x: mydict[x])
travel2020["Country"]=travel2020["Destination"].apply(lambda x: mydict[x])
travel2019["Country"]=travel2019["Destination"].apply(lambda x: mydict[x])

In [72]:
#Double check
df.sample(20)

,DocID,FilerName,MemberName,State,District,Year,Destination,FilingType,DepartureDate,ReturnDate,TravelSponsor,Country
770,500021351,Paige Hutchinson,"Allred, Colin",TX,32.0,2019.0,"Osaka, Japan",Amendment,5/25/2019,6/1/2019,U.S. Association of Former Members of Congress...,Japan
1541,"Clean Fuels Alliance Foundation,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
956,500021500,Emilio Mendez,"Correa, J. Luis",CA,46.0,2019.0,"New York, NY",Original,5/30/2019,5/31/2019,Recording Industry Association of America,USA
2435,500025767,Tracey Mann,"Mann, Tracey",KS,1.0,2023.0,Guatemala,Amendment,8/17/2022,8/20/2022,Guatemala Prospera,Guatemala
482,500026277,Benjamin DeMarzo,"Miller, Mary",IL,15.0,2023.0,"Hot Springs, VA",Original,5/3/2023,5/5/2023,Congressional Institute,USA
628,500024566,Brent Robinson,"Wittman, Robert",VA,1.0,2022.0,"Williamsburg, VA",Original,6/1/2022,6/3/2022,Congressional Institute,USA
758,500026556,Austin Appleby,"Rosendale, Matthew",MT,2.0,2023.0,"Cambridge, MD",Original,6/26/2023,6/28/2023,"Conservative Partnership Institute, Inc.",USA
2330,500025664,Ron Estes,"Estes, Ron",KS,4.0,2022.0,Spain,Original,11/9/2022,11/14/2022,Ripon Society and the Franklin Center for Glob...,Spain
1754,"Clean Fuels Alliance Foundation,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,500023128,Don Bacon,"Bacon, Don",NE,2.0,2021.0,"El Paso, TX",Original,5/20/2021,5/22/2021,Republican Main Street Partnership,USA


In [68]:
df.to_csv("Five Years of Travel.csv")

In [80]:
travel2023["Country"].value_counts()

USA                               1061
Israel                             143
Colombia                            34
United Kingdom                      33
France                              28
Japan                               27
Kenya                               24
United Arab Emirates                23
Norway                              19
Poland                              17
Chile                               16
Brazil                              15
Italy                               15
Jordan                              12
Canada                              11
Lithuania                           11
South Korea                         11
Rwanda                               9
Ukraine                              7
Mozambique                           7
Moldova                              7
Qatar                                5
Tanzania                             4
Nepal                                4
Madagascar                           4
Zambia                   

## Now for the hard part. We're going to scrape every international trip to find out how much they spent.
We only need ~5 years for this analysis, which should be the past 7600 trips or so.

In [19]:
#Download and scrape the pdf of a privately-sponsored congressional trip
def read_housefiles(file):
    '''
    Takes a house file(pdf) and reads the entries for traveler cost and accompanying 
    family member cost. Returns "error" if the file cannot be read.
    '''
    print("Reading PDF")
    doc=convert_from_path(file)
    text=pytesseract.image_to_string(doc[1])
    lines=text.split('\n')
    try:
        linebase=lines.index('and description)')
        traveler=lines[linebase+1]
        accompanying=lines[linebase+2]
        if "$" in traveler and "$" in accompanying: #We know there are dollar signs in the correct line
            return traveler, accompanying
        else:
            try:
                costs=sandwich(text,"and description)\n","Family Member")
                [traveler, accompanying]=costs.split("Accompanying")
                return traveler, accompanying
            except:
                return "error","error"
    except:
        return "wrong page","wrong page"
            


In [16]:
def cleancosts(string):
    '''
    Takes the string returned by the previous function. Attempts to strip out dollar signs
    and other mischellanea to add up the total cost of a trip. 
    If there's still leftover text, it returns the original string
    '''
    string=str(string)
    newstring=string.strip().replace("Traveler ","").replace("|","").replace("/","").replace("\\","")
    newstring=newstring.replace(" Accompanying ","").replace("1$","")
    newstring=newstring.replace("Accompanying ","")
    newstring=newstring.replace("$","").replace(",","").replace("\n","")
    numblist=newstring.split(" ")
    try:
        numbers=[float(x) for x in numblist]
        return sum(numbers)
    except: 
        return newstring

In [17]:
#row=df.loc[0]
#url=row["hrefs"]
#if row["Country"]!="USA":
#    filename=f"House Files/0{row['Traveler']}{row['Dates']}.pdf"
#    response=requests.get(url)
#    print(f"now downloading information on {row['Traveler']}'s trip to {destination}' on {row['Dates']}")
#    wget.download(url, out=filename)
#    [traveler, accompanying]=read_housefiles(filename)
#        df.loc[i, "Traveler Raw Cost"]=traveler
#        df.loc[i,"Accompanying Traveler Raw Cost"]=accompanying
#        print(f"{row['Traveler']}'s trip cost {traveler},{accompanying}\n")
#        os.remove(filename)
#   except:
#        print(f"broken link for {row['Traveler']} on {row['Dates']}")
#        broken_links.append({"row":i, "traveler":row['Traveler'],"Date":row["Dates"],"url":url})


In [20]:
broken_links=[]
for i in range(0, 7600):
    row=df.loc[i]
    destination=row['Destination']
        #docfile=row["hrefs"]
    url=row["hrefs"]
    if destination!="USA":
        filename=f"Downloads/{i}{row['Traveler']}{row['Dates']}.pdf"
        response=requests.get(url)
        if response.status_code<400:
            print(f"now downloading information on {row['Traveler']}'s trip to {destination}' on {row['Dates']}")
            wget.download(url, out=filename)
            [traveler, accompanying]=read_housefiles(filename)
            df.loc[i, "Traveler Raw Cost"]=traveler
            df.loc[i,"Accompanying Traveler Raw Cost"]=accompanying
            print(f"{row['Traveler']}'s trip cost {traveler},{accompanying}\n")
            os.remove(filename)
        else:
            print(f"failed to read {row['Traveler']} on {row['Dates']}")
            broken_links.append({"row":i, "traveler":row['Traveler'],"Date":row["Dates"],"url":url})
    time.sleep(1)
print("Scraping complete")

now downloading information on Vivian Moeglein's trip to united arab emirates' on Dec. 6, 2023 - Dec. 12, 2023
100% [....................................................] 23790435 / 23790435Reading PDF
Vivian Moeglein's trip cost 
$1,738*

$1,702.90 *Averaged increased

, |aFlights & local


now downloading information on Aniela Butler's trip to united arab emirates' on Dec. 6, 2023 - Dec. 12, 2023
100% [......................................................] 1473254 / 1473254Reading PDF
Aniela Butler's trip cost error,error

now downloading information on Brandon Mooney's trip to united arab emirates' on Dec. 6, 2023 - Dec. 11, 2023
100% [......................................................] 2093417 / 2093417Reading PDF
Brandon Mooney's trip cost error,error

failed to read Craig Wheeler on Dec. 6, 2023 - Dec. 11, 2023
now downloading information on Julia Letlow's trip to united arab emirates' on Dec. 7, 2023 - Dec. 11, 2023
100% [....................................................

100% [......................................................] 2197294 / 2197294Reading PDF


Jessica Larsen's trip cost 
Traveler .

$3000 per person forounc-p, | S230PP per night — | $355pp total N/A

flight from Washington, DC - $1 .150pp total for 5

Seoul, South Korea .

nights

,


now downloading information on Max Price's trip to south korea' on Oct. 7, 2023 - Oct. 13, 2023
100% [......................................................] 2153006 / 2153006Reading PDF
Max Price's trip cost 
Traveler .

$3000 per person forounc-p, | S230PP per night — | $355pp total N/A

flight from Washington, DC - $1 .150pp total for 5

Seoul, South Korea .

nights

,


now downloading information on Monika Konrad's trip to south korea' on Oct. 7, 2023 - Oct. 13, 2023
100% [......................................................] 2364129 / 2364129Reading PDF
Monika Konrad's trip cost 
Traveler .

$3000 per person forounc-p, | S230PP per night — | $355pp total N/A

flight from Washington, DC - $1 .150pp total for 5

Seoul, South Korea .

nights

,


now downloading information on Elizabeth Mo

Glenn Ivey's trip cost Traveler $19,290.21 $1,911.70 $1,430.13 $4,651.21,Accompanying 1$16,840.21 $1,911.70 $1,430.13 $4,651.21

now downloading information on Glenn Ivey's trip to rwanda' on Sept. 1, 2023 - Sept. 10, 2023
100% [......................................................] 1179617 / 1179617Reading PDF
Glenn Ivey's trip cost Traveler $19,290.21 $1,911.70 $1,430.13 $4,651.21,Accompanying 1$16,840.21 $1,911.70 $1,430.13 $4,651.21

now downloading information on Jim Costa's trip to ukraine' on Sept. 6, 2023 - Sept. 10, 2023
100% [......................................................] 1804825 / 1804825Reading PDF
Jim Costa's trip cost Traveler 16 403 USD 809 USD 61 USD 376 USD -
conference
participation
,


now downloading information on Jonathan L. Jackson's trip to israel' on Sept. 1, 2023 - Sept. 10, 2023
100% [......................................................] 1310960 / 1310960Reading PDF
Jonathan L. Jackson's trip cost Traveler $19,290.21 $3,221.80 $1,430.13 $4,651.21


now downloading information on James Dolbow's trip to poland' on Aug. 27, 2023 - Sept. 2, 2023
100% [......................................................] 1240807 / 1240807Reading PDF
James Dolbow's trip cost 
Traveler $2387.05 $573.48 $570.68 $8.19 Warsaw Uprising
Museum tour
Private transportation
$8.26 Museum of
, NIA N/A Qpcupation and Freedom


now downloading information on Christopher Ringer's trip to lithuania,' on Aug. 27, 2023 - Sept. 2, 2023
100% [....................................................] 10180421 / 10180421Reading PDF
Christopher Ringer's trip cost 
$2387.05 $573.48 $570.68 $8.19 Warsaw Uprising
Museum tour
Private transportation
$8.26 Museum of
, Qpxupation and Freedom


now downloading information on Christopher Ringer's trip to poland' on Aug. 27, 2023 - Sept. 2, 2023
100% [....................................................] 10180421 / 10180421Reading PDF
Christopher Ringer's trip cost 
$2387.05 $573.48 $570.68 $8.19 Warsaw Uprising
Museum tour
Private tr

...

 49% [.........................                           ]  5267456 / 10586394

 71% [.....................................               ]  7593984 / 10586394

 94% [.................................................   ] 10043392 / 10586394

100% [....................................................] 10586394 / 10586394Reading PDF
Elizabeth Bell's trip cost wrong page,wrong page

now downloading information on Conor Lamb's trip to germany' on Oct. 8, 2022 - Oct. 15, 2022
100% [........................................................] 858714 / 858714Reading PDF
Conor Lamb's trip cost $7678.89 $1,508.32 $696.67 _-
Traveler
, |NA NA NA NA


now downloading information on Elizabeth Bell's trip to united kingdom' on Oct. 9, 2022 - Oct. 15, 2022
100% [....................................................] 10586394 / 10586394Reading PDF
Elizabeth Bell's trip cost wrong page,wrong page

now downloading information on Conor Lamb's trip to united kingdom' on Oct. 8, 2022 - Oct. 15, 2022
100% [........................................................] 858714 / 858714Reading PDF
Conor Lamb's trip cost $7678.89 $1,508.32 $696.67 _-
Traveler
, |NA NA NA NA


now downloading information on Heather Painter's trip to germany' on Oct. 9, 2022

IndexError: list index out of range

In [21]:
i

2012

In [22]:
df

,Unnamed: 0,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Country,Traveler Raw Cost,Accompanying Traveler Raw Cost
0,0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,"\n$1,738*\n\n$1,702.90 *Averaged increased\n\n",|aFlights & local\n
1,1,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,error,error
2,2,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,error,error
3,3,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,NaN,NaN
4,4,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,"\nTraveler $13,703.03 $2,222.00 $451.91\n$13,0...",\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30045,45,"Sept. 30, 2007 - Oct. 1, 2007",Jeff Hild,Pete Stark,D,CA-13,"Baltimore, MD",National Helath Policy Forum,Amendment,https://disclosures-clerk.house.gov/gtimages/S...,USA,NaN,NaN
30046,46,"Sept. 27, 2007 - Sept. 30, 2007",Laurent Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, Massachusetts",BMW Foundation Herbert Quandt,Amendment,https://disclosures-clerk.house.gov/gtimages/S...,USA,NaN,NaN
30047,47,"Sept. 27, 2007 - Sept. 30, 2007",Jasper Pedersen,Robert Wexler,D,FL-19,"Boston, MA",BMW Foundation herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,NaN,NaN
30048,48,"Sept. 27, 2007 - Sept. 30, 2007",Laurant Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, MA",BMW Foundation Herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,NaN,NaN


In [23]:
df.to_csv("Partial scrape with costs2.csv")